In [1]:
!pip install --upgrade keras_applications==1.0.6 keras==2.2.4

    100% |████████████████████████████████| 51kB 66.6MB/s ta 0:00:01
    100% |████████████████████████████████| 317kB 95.4MB/s ta 0:00:01
    100% |████████████████████████████████| 2.9MB 99.6MB/s eta 0:00:01�█████▎                    | 1.0MB 79.4MB/s eta 0:00:01
    100% |████████████████████████████████| 20.4MB 100.3MB/s ta 0:00:01      | 2.6MB 61.6MB/s eta 0:00:01��████▎                    | 7.2MB 89.6MB/s eta 0:00:01�██              | 11.5MB 89.9MB/s eta 0:00:01███████████████▊       | 15.7MB 87.3MB/s eta 0:00:01
    100% |████████████████████████████████| 51kB 58.6MB/s ta 0:00:01
    100% |████████████████████████████████| 25.2MB 66.1MB/s ta 0:00:011.1MB 89.0MB/s eta 0:00:01s eta 0:00:01           | 11.0MB 91.2MB/s eta 0:00:01   | 14.8MB 90.8MB/s eta 0:00:01███████████████████▊       | 19.5MB 91.3MB/s eta 0:00:01
    100% |████████████████████████████████| 266kB 95.5MB/s ta 0:00:01
  Running setup.py bdist_wheel for pyyaml ... done
  Stored in directory: /home/ma-user/.cache/pip/

In [2]:
import os
if os.path.exists('./data') == False:
    from modelarts.session import Session
    session = Session()
    
    if session.region_name == 'cn-north-1':
        bucket_path = 'modelarts-labs/end2end/image_recognition/dog_and_cat_25000.tar.gz'
    elif session.region_name == 'cn-north-4':
        bucket_path = 'modelarts-labs-bj4/end2end/image_recognition/dog_and_cat_25000.tar.gz'
    else:
        print('请更换到北京1或北京4')
    
    session.download_data(bucket_path=bucket_path,
                         path = './dog_and_cat_25000.tar.gz')
    !tar xf ./dog_and_cat_25000.tar.gz
    !rm xf ./dog_and_cat_25000.tar.gz

Successfully download file modelarts-labs/end2end/image_recognition/dog_and_cat_25000.tar.gz from OBS to local ./dog_and_cat_25000.tar.gz
rm: cannot remove 'xf': No such file or directory


In [3]:
!mkdir model

In [4]:
from PIL import Image
import os
import numpy as np
def load_data():
    dirname = './data'
    path = './data'
    num_train = 25000
    x_train = np.empty((num_train,224,224,3),dtype='uint8')
    y_train = np.empty((num_train,1),dtype='uint8') ##生成指定形状的数组
    index = 0
    for file in os.listdir('./data'):
        image = Image.open(os.path.join(dirname,file)).resize((224,224))
        image = np.array(image)
        x_train[index,:,:,:] = image
        if 'cat' in file:
            y_train[index,0] = 1
        elif 'dog' in file:
            y_train[index,0] = 0
            
        index += 1
    return (x_train,y_train)

In [5]:
x_train,y_train = load_data()
print(x_train.shape)
print(y_train.shape)

(25000, 224, 224, 3)
(25000, 1)


In [6]:
from keras.utils import np_utils
def process_data(x_train,y_train):
    x_train = x_train.astype(np.float32)
    x_train = x_train / 255.0
    n_class = 2
    y_train = np_utils.to_categorical(y_train,n_class)
    return x_train,y_train

Using TensorFlow backend.
/home/ma-user/anaconda3/envs/TensorFlow-1.13.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ma-user/anaconda3/envs/TensorFlow-1.13.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ma-user/anaconda3/envs/TensorFlow-1.13.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16",

In [7]:
x_train,y_train = process_data(x_train,y_train)
y_train.shape

(25000, 2)

In [8]:
from keras.models import Model
from keras.layers import Dense,GlobalAveragePooling2D
def bulid_model(base_model):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(2,activation='softmax')(x)
    model = Model(inputs=base_model.input,output=predictions)
    return model

In [9]:
from keras.applications.vgg16 import VGG16
base_model = VGG16(weights=None,include_top=False)

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
import keras
model = bulid_model(base_model)
opt = keras.optimizers.rmsprop(lr=0.0001)

/home/ma-user/anaconda3/envs/TensorFlow-1.13.1/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  import sys


In [11]:
model.compile(loss='binary_crossentropy',
               optimizer=opt,
               metrics=['accuracy'])

In [12]:
import keras.backend as K
def schd(epoch):
    decay = 1e-6
    lr = K.get_value(model.optimizer.lr)
    K.set_value(model.optimizer.lr,lr-decay/(epoch+1))
    print('lr changed to {}'.format(lr))
    print('lll:',K.get_value(model.optimizer.lr))
    return K.get_value(model.optimizer.lr)

In [13]:
from keras.callbacks import LearningRateScheduler,EarlyStopping,ReduceLROnPlateau,ModelCheckpoint
es = EarlyStopping(monitor='val_acc',min_delta=0.001,patience=5,verbose=1,mode='auto')
cp = ModelCheckpoint(filepath='./model/ckp_vgg16_dog_and_cat.h5',monitor='val_acc',
                    verbose=1,save_best_only=True,mode='auto',period=1)
lr = ReduceLROnPlateau(monitor='val_acc',factor=0.1,patience=5,verbose=1,mode='auto',min_lr=0)
ls = LearningRateScheduler(schedule=schd,verbose=1)
callback = [es,cp,lr,ls]

In [14]:
history = model.fit(x = x_train,
                     y = y_train,
                     batch_size=16,
                     epochs=5,
                     verbose=1,
                     callbacks=callback,
                     validation_split=0.25,
                     shuffle=True,
                     initial_epoch=0)

Instructions for updating:
Use tf.cast instead.


Train on 18750 samples, validate on 6250 samples
Epoch 1/5
lr changed to 9.999999747378752e-05
lll: 9.9e-05

Epoch 00001: LearningRateScheduler setting learning rate to 9.9e-05.
18750/18750 [==============================] - 178s 10ms/step - loss: 0.6751 - acc: 0.5868 - val_loss: 0.6210 - val_acc: 0.6610

Epoch 00001: val_acc improved from -inf to 0.66096, saving model to ./model/ckp_vgg16_dog_and_cat.h5
Epoch 2/5
lr changed to 9.899999713525176e-05
lll: 9.85e-05

Epoch 00002: LearningRateScheduler setting learning rate to 9.85e-05.
18750/18750 [==============================] - 161s 9ms/step - loss: 0.6025 - acc: 0.6850 - val_loss: 0.5465 - val_acc: 0.7371

Epoch 00002: val_acc improved from 0.66096 to 0.73712, saving model to ./model/ckp_vgg16_dog_and_cat.h5
Epoch 3/5
lr changed to 9.850000060396269e-05
lll: 9.816667e-05

Epoch 00003: LearningRateScheduler setting learning rate to 9.816667e-05.
18750/18750 [==============================] - 161s 9ms/step - loss: 0.5398 - acc: 0.7395 